In [ ]:
!pip install transformers
# !pip install datasets
# !pip install PyGithub
# !pip install dagshub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from pathlib import Path
from typing import List

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import BertTokenizer, TFBertModel

# to mount google drive and access NLU-datasets
from google.colab import drive

# imports from NLU-datasets
import tensorflow as tf

from keras.utils.vis_utils import plot_model
from keras.models import Sequential, Model
from keras.layers import (Dense, Activation, Conv1D, MaxPooling1D, 
                          Dropout, Flatten, Input, Add, Average,
                          LSTM, Lambda, AveragePooling1D,
                          Reshape)
from tensorflow.keras.optimizers import SGD, Adam
from keras import regularizers
from keras import backend as K
from sklearn.model_selection import StratifiedKFold

Mount google drive to access NLU-datasets https://github.com/tilde-nlp/NLU-datasets that I presaved in my google drive in NLU-datasets directory. This will trigger a pop up window that asks for permission to access google drive.

TODO: Upload it to huggingface and use directly from there https://huggingface.co/docs/datasets/upload_dataset (the dataset has Creative Commons Attribution-ShareAlike 4.0 International License, which allows it)

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if "NLU-datasets" not in os.getcwd():
  os.chdir("drive/My Drive/NLU-datasets")

In [ ]:
!ls

askubuntu  chatbot  README.md  webapps


In [ ]:
def get_data(path: str) -> List[str]:
  with open(path) as f:
    array = []
    for line in list(f):
      array.append(line.split('\n')[0])
    return array


pathlist = Path("chatbot").glob("**/*.txt")

for path in pathlist:
  # because path is object not string
  path_in_str = str(path)
  print(path_in_str)
  if path_in_str == "chatbot/chatbot_train_ans.txt":
    chatbot_train_ans = get_data(path_in_str)
  elif path_in_str == "chatbot/chatbot_test_ans.txt":
    chatbot_test_ans  = get_data(path_in_str)
  elif path_in_str == "chatbot/en/chatbot_test_q.txt":
    en_test  = get_data(path_in_str)
  elif path_in_str == "chatbot/en/chatbot_train_q.txt":
    en_train  = get_data(path_in_str)
  elif path_in_str == "chatbot/lv/chatbot_test_q.txt":
    lv_test  = get_data(path_in_str)
  elif path_in_str == "chatbot/lv/chatbot_train_q.txt":
    lv_train  = get_data(path_in_str)

print(chatbot_train_ans)
print(chatbot_test_ans)
print(en_test)

chatbot/chatbot_train_ans.txt
chatbot/chatbot_test_ans.txt
chatbot/en/chatbot_test_q.txt
chatbot/en/chatbot_train_q.txt
chatbot/lv/chatbot_test_q.txt
chatbot/lv/chatbot_train_q.txt
chatbot/et/chatbot_train_q.txt
chatbot/et/chatbot_test_q.txt
chatbot/ru/chatbot_train_q.txt
chatbot/ru/chatbot_test_q.txt
chatbot/lt/chatbot_test_q.txt
chatbot/lt/chatbot_train_q.txt
['FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'DepartureTime', 'FindConnection', 'FindConnection', 'DepartureTime', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'DepartureTime', 'DepartureTime', 'DepartureTime', 'DepartureTime', 'DepartureT

what tokenizer does? what model does?

enconded_input shape changes by increasing word count: 4 words: (1, 6), 5 words: (1, 9); 7 words: (1, 13) what shape should go into classifier? sentence vector or individual word vectors? what do input_ids, token_type_ids and attention_mask mean in encoded_input? which one should i pass to classifier? probably input_ids because its the only one different from 0 and 1


In [ ]:
model_name = "bert-base-multilingual-cased"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
encoded_input = tokenizer(en_test[0], return_tensors='tf')
output = model(encoded_input)

print(encoded_input)
print(output)

{'input_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=
array([[  101,   177, 21528, 10114, 11783, 24538, 10136, 20732,   102]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 9, 768), dtype=float32, numpy=
array([[[-0.0801831 ,  0.03810783,  0.27046812, ...,  0.39124072,
         -0.11388044, -0.04561077],
        [-0.13535482, -0.61088157,  0.83754325, ...,  0.6064948 ,
         -0.36229783,  0.06997249],
        [-0.9340398 , -0.27897775,  0.34910437, ...,  1.0008146 ,
         -0.27053565,  0.39220393],
        ...,
        [-0.64944226,  0.01610355,  0.06948492, ...,  0.9110723 ,
         -0.3397322 , -0.44170964],
        [-0.43947974,  0.11158036,  0.09911749, ...,  0.7545632 ,
          0

In [ ]:
encoded_input['input_ids']

<tf.Tensor: shape=(1, 9), dtype=int32, numpy=
array([[  101,   177, 21528, 10114, 11783, 24538, 10136, 20732,   102]],
      dtype=int32)>

In [ ]:
encoded_input['token_type_ids']

<tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>

In [ ]:
encoded_input['attention_mask']

<tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>

In [ ]:
output["last_hidden_state"]

<tf.Tensor: shape=(1, 9, 768), dtype=float32, numpy=
array([[[-0.0801831 ,  0.03810783,  0.27046812, ...,  0.39124072,
         -0.11388044, -0.04561077],
        [-0.13535482, -0.61088157,  0.83754325, ...,  0.6064948 ,
         -0.36229783,  0.06997249],
        [-0.9340398 , -0.27897775,  0.34910437, ...,  1.0008146 ,
         -0.27053565,  0.39220393],
        ...,
        [-0.64944226,  0.01610355,  0.06948492, ...,  0.9110723 ,
         -0.3397322 , -0.44170964],
        [-0.43947974,  0.11158036,  0.09911749, ...,  0.7545632 ,
          0.0909415 , -0.18004197],
        [-0.285816  , -0.20664008,  1.011271  , ...,  0.43730527,
         -0.19045143,  0.07825344]]], dtype=float32)>

In [ ]:
output["last_hidden_state"][-1][-1]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([-2.85816014e-01, -2.06640080e-01,  1.01127100e+00,  1.69525176e-01,
        4.14485633e-01,  4.47487175e-01, -8.07733178e-01, -3.04685310e-02,
        6.30859196e-01,  1.63276196e-01,  4.42684628e-02,  2.33403414e-01,
        2.33043224e-01,  9.90195721e-02,  7.55262300e-02, -3.19861114e-01,
        1.78901434e-01,  1.21485806e+00,  4.84490991e-02,  3.46210748e-01,
        1.32622540e-01, -1.11739404e-01, -6.08280838e-01, -8.09180886e-02,
        6.48078442e-01, -8.46020579e-02, -6.56183600e-01,  5.01036942e-01,
       -3.52555335e-01, -1.67581841e-01,  1.31201386e-01,  1.03604302e-01,
        6.91107363e-02, -2.77195930e-01,  3.67443085e-01,  1.57708913e-01,
       -2.31118336e-01, -2.12883919e-01, -2.05561489e-01, -3.60636055e-01,
       -5.36699593e-01,  3.67783636e-01,  5.39841056e-02,  1.12445541e-02,
        8.39076579e-01, -2.81842649e-01,  3.12413156e-01,  2.32986987e-01,
        4.03369904e-01, -6.85853839e-01,  4.51510876

**batch_size** -  number of data points sampled to calculate the gradient on each iteration

large vs small batch size performance table2 https://arxiv.org/pdf/1609.04836.pdf

**sequence_length** -  number of tokens (words) in a sentence. causes the layer to pad or truncate sequences to exactly sequence_length values. how to handle variable-length inputs (sentences of different lengths)?

**hidden_size** (int, optional, defaults to 768) — Dimensionality of the encoder layers and the pooler layer.

**last_hidden_state** (tf.Tensor of shape (batch_size, sequence_length, hidden_size)) — Sequence of hidden-states at the output of the last layer of the model.


**pooler_output** (tf.Tensor of shape (batch_size, hidden_size)) — Last layer hidden-state of the first token of the sequence (classification token) further processed by a Linear layer and a Tanh activation function. The Linear layer weights are trained from the next sentence prediction (classification) objective during pretraining.

This output is usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence.

https://huggingface.co/docs/transformers/main_classes/output#transformers.modeling_outputs.BaseModelOutputWithPooling

In [ ]:
# which one (last_hidden_state or pooler_output) should be passed to classifier?
# i guess try both and see how it works
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 9, 768), dtype=float32, numpy=
array([[[-0.0801831 ,  0.03810783,  0.27046812, ...,  0.39124072,
         -0.11388044, -0.04561077],
        [-0.13535482, -0.61088157,  0.83754325, ...,  0.6064948 ,
         -0.36229783,  0.06997249],
        [-0.9340398 , -0.27897775,  0.34910437, ...,  1.0008146 ,
         -0.27053565,  0.39220393],
        ...,
        [-0.64944226,  0.01610355,  0.06948492, ...,  0.9110723 ,
         -0.3397322 , -0.44170964],
        [-0.43947974,  0.11158036,  0.09911749, ...,  0.7545632 ,
          0.0909415 , -0.18004197],
        [-0.285816  , -0.20664008,  1.011271  , ...,  0.43730527,
         -0.19045143,  0.07825344]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[ 0.24232776, -0.05618617,  0.26164398, -0.18765198, -0.06455288,
         0.46789142,  0.22187352,  0.2141517 , -0.4258368 ,  0.29156756,
        -0.11752241, -0.1815

In [ ]:
max_features = 10000
embedding_dim = 16

In [ ]:
# model = tf.keras.Sequential([
#   layers.Embedding(max_features + 1, embedding_dim),
#   layers.Dropout(0.2),
#   layers.GlobalAveragePooling1D(),
#   layers.Dropout(0.2),
#   layers.Dense(1)])

# model.summary()
